In [1]:
# django shell 환경설정
import os
import django
os.environ['DJANGO_SETTINGS_MODULE'] = 'config.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

django.setup()

# Paginator 클래스
- 전체 페이징 처리를 관리하는 클래스
- 전체 데이터관련 정보, 각 페이지당 보여줄 데이터의 정보 등을 제공
- ListView에서 paginate_by를 설정하면 자동으로 생성되어 context data에 저장된다.

# Page 클래스
- 한페이지에대한 데이터를 관리
- Paginator가 생성해서 제공.
- iterable 타입. 원소로 데이터를 제공
- Page객체.object_list 속성: 페이지가 가진 데이터들을 List로 반환

In [6]:
data_list = list("0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ")

62

In [7]:
from django.core.paginator import Paginator

In [8]:
# paginator 생성
pn = Paginator(data_list, 5)  # 전체데이터목록, 한페이지당 데이터수(paginate_by설정한 값)

In [12]:
# paginator속성
print('전체 데이터 수:', pn.count)
print('총 페이지 수:', pn.num_pages)
print("시작페이~끝페이지+1:", pn.page_range) #시작페이지~끝페이지의 범위를 range객체로 반환

전체 데이터 수: 62
총 페이지 수: 13
시작페이,끝페이지+1: range(1, 14)


In [13]:
for p in pn.page_range:
    print(p, end='\t')

1	2	3	4	5	6	7	8	9	10	11	12	13	

In [15]:
# Paginator에서 특정 페이지정보가지는 Page 객체를 조회
# Page - iterable ->그 페이지의 데이터를 반환
page1 = pn.page(1) #pn.page(페이지번호)
print(type(page1), page1)
print(pn.page(10))

<class 'django.core.paginator.Page'> <Page 1 of 13>
<Page 10 of 13>


In [17]:
# pn.page(50) #없는 페이지 조회시 EmptyPage Exception발생

In [20]:
for data in page1:
    print(data)

0
1
2
3
4


In [21]:
for data in pn.page(10):
    print(data)

J
K
L
M
N


In [22]:
# 페이지가 가진 데이터(원소)들을 List 반환
page1_list = page1.object_list
print(page1_list)

['0', '1', '2', '3', '4']


In [ ]:
# 함수형 View로 List 처리 구현
# url?page=번호  : 페이지번호 querystring = 요청파라미터
def list_view(request):
    page_no = request.GET.get('page', 1)
    pn = Paginator(Model.objects.all(), 10)
    page = pn.page(page_no)
    # + 페이지링크를 위한 값들 조회->context_data로 전달.
    return render(request, '템플릿 경로', {'object_list':page})


## 이전/다음 페이지 유무
- Page.has_previous() / Page.has_next()
- 1페이지: 이전페이지? X, 다음페이지? O
- 13페이지: 이전? O, 다음페이지? X
- 10페이지: 이전페이지? O, 다음페이지? O

In [23]:
page1 = pn.page(1)
page10 = pn.page(10)
page13 = pn.page(13)

In [28]:
print('page1', page1.has_previous(), page1.has_next(), sep=' - ')
print('page10', page10.has_previous(), page10.has_next(), sep=' - ')
print('page13', page13.has_previous(), page13.has_next(), sep=' - ')

page1 - False - True
page10 - True - True
page13 - True - False


## 현재/이전/다음페이지 번호 조회
- Page.number : 현재 페이지 번호
- Page.previous_page_number(): 이전 페이지 번호
- Page.next_page_number(): 다음 페이지 번호

In [33]:
print("페이지번호:", page1.number)
if page1.has_previous():
    print("이전페이지번호:", page1.previous_page_number())
else:
    print("첫번째 페이지입니다.")

if page1.has_next():
    print('다음페이지번호:', page1.next_page_number())
else:
    print("마지막 페이지입니다.")

페이지번호: 1
첫번째 페이지입니다.
다음페이지번호: 2


In [34]:
print("페이지번호:", page10.number)
if page10.has_previous():
    print("이전페이지번호:", page10.previous_page_number())
else:
    print("첫번째 페이지입니다.")

if page10.has_next():
    print('다음페이지번호:', page10.next_page_number())
else:
    print("마지막 페이지입니다.")

페이지번호: 10
이전페이지번호: 9
다음페이지번호: 11


### 각 페이지별 데이터 조회

In [35]:
data_list # Post.objects.all() - 원소1개 Post객체 1개

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z']

In [39]:
# template
pn = Paginator(data_list, 10) #paginate_by=10
for page_no in pn.page_range:
    print(f"{page_no} 페이지")
    for data in pn.page(page_no):
        print(data, end=', ')
    print('\n============================================')

1 페이지
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 
2 페이지
a, b, c, d, e, f, g, h, i, j, 
3 페이지
k, l, m, n, o, p, q, r, s, t, 
4 페이지
u, v, w, x, y, z, A, B, C, D, 
5 페이지
E, F, G, H, I, J, K, L, M, N, 
6 페이지
O, P, Q, R, S, T, U, V, W, X, 
7 페이지
Y, Z, 


# 현재 페이지(요청페이지)가 속한 page 그룹의 (page_range)에서의 시작 index와 끝 index를 조회

In [42]:
pn = Paginator(data_list, 3)
for page_no in pn.page_range:
    print(f'<a href="/board/list?page={page_no}">{page_no}</a>')

<a href="/board/list?page=1">1</a>
<a href="/board/list?page=2">2</a>
<a href="/board/list?page=3">3</a>
<a href="/board/list?page=4">4</a>
<a href="/board/list?page=5">5</a>
<a href="/board/list?page=6">6</a>
<a href="/board/list?page=7">7</a>
<a href="/board/list?page=8">8</a>
<a href="/board/list?page=9">9</a>
<a href="/board/list?page=10">10</a>
<a href="/board/list?page=11">11</a>
<a href="/board/list?page=12">12</a>
<a href="/board/list?page=13">13</a>
<a href="/board/list?page=14">14</a>
<a href="/board/list?page=15">15</a>
<a href="/board/list?page=16">16</a>
<a href="/board/list?page=17">17</a>
<a href="/board/list?page=18">18</a>
<a href="/board/list?page=19">19</a>
<a href="/board/list?page=20">20</a>
<a href="/board/list?page=21">21</a>


In [47]:
# page그룹당 10 페이지번호를 묶는다고 했을때 
# 1 page 그룹의 시작, 끝: 1 ~ 10
pn.page_range[0:10]
# 2 page 글부의 시작, 끝: 11 ~ 20
pn.page_range[10:20]

range(11, 21)

In [48]:
pn.page_range

range(1, 22)

In [62]:
pn = Paginator(data_list, 3)
current_page = 20
page_group_count = 5 # 한페이지 그룹에 속한 페이지수 (페이지를 몇페이지씩 묶을 것인지.)

# 현재페이지가 속한 페이지그룹의 시작 페이지의 index
start_index = int((current_page-1)/page_group_count)*page_group_count
# 현재페이지가 속한 페이지그룹의 마지막 페이지의 index
end_index = start_index + page_group_count
print(start_index, end_index)
print("페이지")
pagegroup_page_range = pn.page_range[start_index:end_index]
print(pagegroup_page_range)

15 20
페이지
range(16, 21)


In [63]:
for page_no in pagegroup_page_range:
     print(f'<a href="/board/list?page={page_no}">{page_no}</a>')

<a href="/board/list?page=16">16</a>
<a href="/board/list?page=17">17</a>
<a href="/board/list?page=18">18</a>
<a href="/board/list?page=19">19</a>
<a href="/board/list?page=20">20</a>


# Post  데이터를 추가

In [2]:
from board.models import Post, Category

In [3]:
cate = Category(pk=1)

In [4]:
# 352
for i in range(352): 
    post = Post(title=f"제목-{i}", content=f"내용{i}\n내용{i}\n내용{i}", category=cate)
    post.save()

In [5]:
Post.objects.all().count()

355